In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', 500)
client = MongoClient ('localhost', 27017)

El json anterior lo he llevado a MongoDB, conectamos con él y comentazomos a trabajar.

In [2]:
collection = client['t_project'].tableau_project

Lo importamos a un dataset

In [3]:
data1 = pd.DataFrame(list(collection.find()))
data1.head()

,Point,_id,category_code,city,country_code,founded_year,latitude,longitude,name,number_of_employees
0,"{'type': 'Point', 'coordinates': [-122.187925,...",5cd5a7372cda8341777639e6,web,Kirkland,USA,2008,47.704875,-122.187925,PokerPlayersSocialNetwork,3
1,"{'type': 'Point', 'coordinates': [-122.393229,...",5cd5a7372cda8341777639e7,games_video,San Francisco,USA,2008,37.781265,-122.393229,Crunchyroll,50
2,"{'type': 'Point', 'coordinates': [10.023246, 5...",5cd5a7372cda8341777639e8,games_video,Norderstedt,DEU,2009,53.707739,10.023246,alluc,7
3,"{'type': 'Point', 'coordinates': [151.039775, ...",5cd5a7372cda8341777639e9,web,Sydney,AUS,2008,-34.054416,151.039775,thatsmymouse,3
4,"{'type': 'Point', 'coordinates': [51.5082, 25....",5cd5a7372cda8341777639ea,games_video,Doha,QAT,2008,25.294800,51.508200,Triptuber,2


Utilizamos la función 'near' para calcular cuantas empresas tenemos a una distacia de 3 kilómetros de cada una de las empresas y agregamos una columna con dicho número.

In [4]:
def compañias_cercanas(Point, max_distance):
    cercanas = pd.DataFrame(list(collection.find(
        {'Point': { '$near': {'$geometry': Point, '$maxDistance': max_distance}}}, {'name': 1})))
    return len(cercanas)

In [5]:
distancia = 3000
data1['numero_compañias_cerca'] = data1['Point'].apply(lambda x: compañias_cercanas(x,distancia))


In [6]:
data1.head()

,Point,_id,category_code,city,country_code,founded_year,latitude,longitude,name,number_of_employees,numero_compañias_cerca
0,"{'type': 'Point', 'coordinates': [-122.187925,...",5cd5a7372cda8341777639e6,web,Kirkland,USA,2008,47.704875,-122.187925,PokerPlayersSocialNetwork,3,2
1,"{'type': 'Point', 'coordinates': [-122.393229,...",5cd5a7372cda8341777639e7,games_video,San Francisco,USA,2008,37.781265,-122.393229,Crunchyroll,50,27
2,"{'type': 'Point', 'coordinates': [10.023246, 5...",5cd5a7372cda8341777639e8,games_video,Norderstedt,DEU,2009,53.707739,10.023246,alluc,7,1
3,"{'type': 'Point', 'coordinates': [151.039775, ...",5cd5a7372cda8341777639e9,web,Sydney,AUS,2008,-34.054416,151.039775,thatsmymouse,3,1
4,"{'type': 'Point', 'coordinates': [51.5082, 25....",5cd5a7372cda8341777639ea,games_video,Doha,QAT,2008,25.294800,51.508200,Triptuber,2,1


In [7]:
data1 = data1.drop(data1[data1['numero_compañias_cerca']<=15].index)
data1.head()

,Point,_id,category_code,city,country_code,founded_year,latitude,longitude,name,number_of_employees,numero_compañias_cerca
1,"{'type': 'Point', 'coordinates': [-122.393229,...",5cd5a7372cda8341777639e7,games_video,San Francisco,USA,2008,37.781265,-122.393229,Crunchyroll,50,27
5,"{'type': 'Point', 'coordinates': [-122.404488,...",5cd5a7372cda8341777639eb,web,San Francisco,USA,2008,37.790032,-122.404488,Unknown Vector,7,26
13,"{'type': 'Point', 'coordinates': [-122.392142,...",5cd5a7372cda8341777639f3,software,San Francisco,USA,2008,37.782263,-122.392142,GoingOn,40,27
19,"{'type': 'Point', 'coordinates': [-122.4042586...",5cd5a7372cda8341777639f9,web,San Francisco,USA,2008,37.803910,-122.404259,ReadBurner,4,19
29,"{'type': 'Point', 'coordinates': [-73.992706, ...",5cd5a7372cda834177763a03,web,New York City,USA,2008,40.755409,-73.992706,Wakozi,5,26


Elimino las empresas que tengan menos de 15 empresas a su alrededor y exporto el dataset resultante en formato csv para poder trabajar con él en Tableau.

In [8]:
data1.to_csv('tableau_project_pt2.csv')